<a href="https://colab.research.google.com/github/sara-yasmin14/ASD-Detection-Using-AI-ML/blob/main/hog_updated_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.feature import hog
from skimage import exposure
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def compute_hog_features(image):
    hog_features, hog_image = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    return hog_features, hog_image

In [ ]:
def save_hog_images_to_excel(directory_path, output_excel_path):
    # Get list of image file names in directory
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Create a new Excel workbook and add a worksheet
    workbook = Workbook()
    worksheet = workbook.active
    worksheet.title = "HOG Images"
    worksheet.append(["Original Image", "HOG Image"])

    for idx, image_file in enumerate(tqdm(image_files, desc="Processing images")):
        # Read image
        image_path = os.path.join(directory_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Compute HOG features and HOG image
        _, hog_image = compute_hog_features(image)

        # Rescale HOG image
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

        # Resize the HOG image to 125x125 using OpenCV
        hog_image_resized = cv2.resize(hog_image_rescaled, (125, 125), interpolation=cv2.INTER_AREA)

        # Save the resized HOG image temporarily
        hog_image_filename = f'temp_hog_image_{idx + 1}.png'
        plt.imsave(hog_image_filename, hog_image_resized, cmap='gray')

        # Add original image name and HOG image to the worksheet
        worksheet.append([image_file])

        # Insert the resized HOG image into the Excel file
        img = ExcelImage(hog_image_filename)
        img.anchor = f'B{idx + 2}'  # Start inserting images from the second row
        worksheet.add_image(img)

    # Save the workbook
    workbook.save(output_excel_path)

    # Clean up temporary images
    for idx in range(len(image_files)):
        os.remove(f'temp_hog_image_{idx + 1}.png')

# Directory containing images (adjust this path based on your directory)
directory_path = '/content/drive/My Drive/DATASET'
# Path to save the Excel file (adjust this path based on your directory)
output_excel_path = '/content/drive/My Drive/HOG_Features/HOG_Images.xlsx'

save_hog_images_to_excel(directory_path, output_excel_path)

Processing images: 100%|██████████| 231/231 [02:09<00:00,  1.78it/s]
